# YOLO: Projeto "Completo" - Análise de Tráfego

In [ ]:
import cv2
from ultralytics import YOLO

In [ ]:
model = YOLO()

In [ ]:
video_path = 'vehicle-counting.mp4'
output_video_path = 'output_traffic_analysis.mp4'

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"Erro: Não foi possível abrir o vídeo em {video_path}")
    exit() # Encerrar se o vídeo não puder ser aberto

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
print(f"Resolução do vídeo: {width}x{height}, FPS: {fps}")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
print(f"Vídeo de saída configurado: {output_video_path}")

In [ ]:
target_classes = {'car': 2, 'truck': 7}
class_counts = {name: 0 for name in target_classes.keys()}

# Definir uma 'linha de contagem' ou 'área de interesse'
count_line_y = int(height * 0.7) # Linha horizontal a 70% da altura da imagem

In [ ]:
frame_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Realizar inferência
    results = model.predict(source=frame, conf=0.25, iou=0.5, verbose=False)

    # Desenhar a linha de contagem no frame
    cv2.line(frame, (0, count_line_y), (width, count_line_y), (255, 0, 255), 2) # Linha magenta

    # Processar cada detecção
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            confidence = float(box.conf[0].cpu().numpy())
            class_id = int(box.cls[0].cpu().numpy())
            class_name = model.names[class_id]

            if class_name in target_classes and confidence > 0.5: # Filtrar por confiança e classes de interesse
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                
                # Plotar a bounding box e label
                color = (0, 255, 0) # Verde
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                label = f'{class_name} {confidence:.2f}'
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
                cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1) # Centro do objeto
                
    current_frame_detections_info = {}
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            class_id = int(box.cls[0].cpu().numpy())
            class_name = model.names[class_id]
            if class_name in target_classes:
                if class_name not in current_frame_detections_info:
                    current_frame_detections_info[class_name] = 0
                current_frame_detections_info[class_name] += 1
                # Se o centro do objeto cruza a linha (simplificado para demo)
                center_y = (y1 + y2) // 2
                if abs(center_y - count_line_y) < 10: # Se estiver na zona de contagem
                    class_counts[class_name] += 1 # Contagem simples (pode haver overcounting)

    # Mostrar contagens no frame
    y_offset = 30
    for class_name, count in class_counts.items():
        text = f'{class_name}: {count}'
        cv2.putText(frame, text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        y_offset += 30
        
    cv2.imshow('Traffic Analysis', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
    out.write(frame)

    frame_idx += 1

In [ ]:
# 6. Finalização e Resultados
print(f"Total de frames processados: {frame_idx}")
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Vídeo de análise salvo como '{output_video_path}'")

print("\nEstatísticas Finais (Contagem Simplificada):")
for class_name, count in class_counts.items():
    print(f"- {class_name}: {count}")
